In [44]:
import os
import rasterio
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam

# Folder paths for spectral, texture, and height images
spectral_folder = r'C:\test_td\230714_msi_chip'
texture_folder = r'C:\test_td\230714_tex_chip'
height_folder = r'C:\test_td\230714_chm_chip'

# Load the Excel file that has corresponding chip names and true tiller density values
excel_file = r'C:\test_td\td_2023.xlsx'
df_tiller_density = pd.read_excel(excel_file)

# List all the chip image filenames from the spectral folder
chip_filenames = df_tiller_density['Chip_Name'].tolist()

# Prepare lists to store features and true tiller density values
input_features_list = []
true_tiller_densities = []

# Prepare a DataFrame to save the features
columns = ['Chip_Name', 'Blue', 'Green', 'Red', 'Red_Edge', 'NIR', 'NDVI', 'GNDVI', 'RVI_1', 'GCI', 'RGVI', 'DVI', 'SAVI', 'MSAVI', 'OSAVI', 'RDVI', 
          'TVI', 'TSAVI', 'PVI', 'SAVI_2', 'ATSAVI', 'NDWI', 'IKAW', 'SRPI', 'RVI_2', 'MCARI', 'MCARI_1', 'MCARI_2', 'MTVI_1', 'MTVI_2',
          'R_MCARI_MTVI2', 'EVI', 'DATT', 'DNCI', 'PSRI', 'SIPI', 'SPVI', 'TCARI', 'R_TCARI_OSAVI', 'RERI', 'NDRE', 'MTCI', 'EVI_2', 'RECI', 'NEXG',
          'NGRDI', 'ENDVI', 'ARI_2', 'CRI_2'] + \
          [f'Tex_{i+1}' for i in range(40)] + \
          ['Hmean', 'Hmedian', 'Hmin', 'Hmax', 'Hstd', 'Hp90', 'Hp93', 'Hp95', 'Hp98', 'Hp99']
features_df = pd.DataFrame(columns=columns)

# Function to compute RMSE
def calculate_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Function to compute NDVI from average NIR and Red band values
def calculate_ndvi(avg_nir, avg_red):
    return (avg_nir - avg_red) / (avg_nir + avg_red)
    
# Function to calculate NDVI (Normalized Difference Vegetation Index)
def calculate_ndvi(avg_nir, avg_red):
    return (avg_nir - avg_red) / (avg_nir + avg_red)

# Function to calculate GNDVI
def calculate_gndvi(avg_nir, avg_green):
    return (avg_nir - avg_green) / (avg_nir + avg_green)

# Function to calculate RVI_1
def calculate_rvi_1(avg_nir, avg_red):
    return avg_nir / avg_red

# Function to calculate GCI
def calculate_gci(avg_nir, avg_green):
    return (avg_nir / avg_green) - 1.0

# Function to calculate RGVI
def calculate_rgvi(avg_red, avg_green):
    return avg_red / avg_green

# Function to calculate DVI
def calculate_dvi(avg_nir, avg_red):
    return avg_nir - avg_red

# Function to calculate SAVI
def calculate_savi(avg_nir, avg_red):
    L = 0.5
    return ((avg_nir - avg_red) / (avg_nir + avg_red + L)) * (1.0 + L)

# Function to calculate MSAVI
def calculate_msavi(avg_nir, avg_red):
    return 0.5 * ((2.0 * avg_nir) + 1.0 - np.sqrt(np.square(2.0 * avg_nir + 1.0) - 8.0 * (avg_nir-avg_red)))

# Function to calculate OSAVI
def calculate_osavi(avg_nir, avg_red):
    return (avg_nir - avg_red) / (avg_nir + avg_red + 0.16)

# Function to calculate RDVI
def calculate_rdvi(avg_nir, avg_red):
    return np.sqrt((np.square(avg_nir - avg_red)) / (avg_nir + avg_red))

# Function to calculate TVI
def calculate_tvi(avg_nir, avg_red, avg_green):
    return 60.0 * (avg_nir - avg_green) - 100.0 * (avg_red - avg_green)

# Function to calculate TSAVI
def calculate_tsavi(avg_nir, avg_red):
    s = 0.33
    a = 0.5
    X = 1.5
    return (s * (avg_nir - s * avg_red - a)) / (a * avg_nir + avg_red - a * s + X * (1 + np.square(s)))

# Function to calculate PVI
def calculate_pvi(avg_nir, avg_red):
    a = 0.3
    b = 0.5
    return (avg_nir - a * avg_red - b) / (np.sqrt(1 + np.square(a)))                 

# Function to calculate SAVI_2
def calculate_savi_2(avg_nir, avg_red):
    a = 0.96916
    b = 0.084726
    return avg_nir / (avg_red + (b / a))

# Function to calculate ATSAVI
def calculate_atsavi(avg_nir, avg_red):
    a = 0.96916
    b = 0.084726
    X = 0.08
    return (a*(-a*avg_red-b))/(a*avg_nir+avg_red-a*b+X*(1+np.square(a)))

# Function to calculate NDWI
def calculate_ndwi(avg_nir, avg_green):
    return (avg_green-avg_nir)/(avg_green+avg_nir)

# Function to calculate IKAW
def calculate_ikaw(avg_red, avg_blue):
    return (avg_red-avg_blue)/(avg_red+avg_blue)

# Function to calculate SRPI
def calculate_srpi(avg_red, avg_blue):
    return avg_blue / avg_red

# Function to calculate RVI_2
def calculate_rvi_2(avg_nir, avg_green):
    return avg_nir / avg_green

# Function to calculate MCARI
def calculate_mcari(avg_red_edge, avg_red, avg_green):
    return (avg_red_edge-avg_red-0.2*(avg_red_edge-avg_green))*(avg_red_edge/avg_red)

# Function to calculate MCARI_1
def calculate_mcari_1(avg_nir, avg_red, avg_green):
    return 1.2*(2.5*(avg_nir-avg_red)-1.3*(avg_nir-avg_green))  

# Function to calculate MCARI_2
def calculate_mcari_2(avg_nir, avg_red, avg_green):
    return 1.5*(2.5*(avg_nir-avg_red)-1.3*(avg_nir-avg_green))*(np.square(2.0*avg_nir+1))-(6.0*avg_nir-5.0*avg_red)-0.5

# Function to calculate MTVI_1
def calculate_mtvi_1(avg_nir, avg_red, avg_green):
    return 1.2*(1.2*(avg_nir-avg_green)-2.5*(avg_red-avg_green))

# Function to calculate MTVI_2
def calculate_mtvi_2(avg_nir, avg_green):
    return (avg_nir - avg_green) / (avg_nir + avg_green)

# Function to calculate R_MCARI_MTVI2
def calculate_r_mcari_mtvi2(avg_nir, avg_red, avg_green):
    return 1.5*(1.2*(avg_nir-avg_green)-2.5*(avg_red-avg_green))*(np.square(2*avg_nir+1))-(6.0*avg_nir-5.0*avg_red)-0.5

# Function to calculate EVI (Enhanced Vegetation Index)
def calculate_evi(avg_nir, avg_red, avg_blue):
    G = 2.5  # Gain factor
    C1 = 6.0  # Coefficient
    C2 = 7.5  # Coefficient
    L = 1.0   # Canopy background adjustment
    return G * ((avg_nir - avg_red) / (avg_nir + C1 * avg_red - C2 * avg_blue + L))

# Function to calculate DATT
def calculate_datt(avg_nir, avg_red_edge, avg_red):
    return (avg_nir-avg_red_edge)/(avg_nir-avg_red)

# Function to calculate DNCI
def calculate_dnci(avg_red_edge, avg_green):
    return (avg_red_edge-avg_green)/(avg_red_edge+avg_green)

# Function to calculate PSRI
def calculate_psri(avg_red_edge, avg_red, avg_green):
    return (avg_red-avg_green)/avg_red_edge

# Function to calculate SIPI
def calculate_sipi(avg_nir, avg_red, avg_blue):
    return (avg_nir-avg_blue)/(avg_nir+avg_red)

# Function to calculate SPVI
def calculate_spvi(avg_nir, avg_red, avg_green):
    return 0.4*3.7*(avg_nir-avg_red)-1.2*np.absolute(avg_green-avg_red)

# Function to calculate TCARI
def calculate_tcari(avg_red_edge, avg_red, avg_green):
    return 3.0*((avg_red_edge-avg_red)-0.2*(avg_red_edge-avg_green)*(avg_red_edge/avg_red))

# Function to calculate R_TCARI_OSAVI
def calculate_r_tcari_osavi(avg_nir, avg_red_edge, avg_red, avg_green):
    return (3.0*((avg_red_edge-avg_red)-0.2*(avg_red_edge-avg_green)*(avg_red_edge/avg_red)))/((avg_nir-avg_red)/(avg_nir+avg_red+0.16))

# Function to calculate RERI
def calculate_reri(avg_nir, avg_red_edge, avg_red):
    return (avg_red_edge-avg_red)/avg_nir

# Function to calculate NDRE
def calculate_ndre(avg_nir, avg_red_edge):
    return (avg_nir-avg_red_edge)/(avg_nir+avg_red_edge)

# Function to calculate MTCI
def calculate_mtci(avg_nir, avg_red_edge, avg_red):
    return (avg_nir-avg_red_edge)/(avg_red_edge-avg_red)

# Function to calculate EVI_2
def calculate_evi_2(avg_nir, avg_red):
    return 2.5*((avg_nir-avg_red)/(avg_nir+2.4*avg_red+1.0))

# Function to calculate RECI
def calculate_reci(avg_nir, avg_red_edge):
    return (avg_nir/avg_red_edge)-1

# Function to calculate NEXG
def calculate_nexg(avg_red, avg_green, avg_blue):
    return (2*avg_green-avg_red-avg_blue)/(avg_green+avg_red+avg_blue)

# Function to calculate NGRDI
def calculate_ngrdi(avg_red, avg_green):
    return (avg_green-avg_red)/(avg_green+avg_red)

# Function to calculate ENDVI
def calculate_endvi(avg_nir, avg_green, avg_blue):
    return (avg_nir+avg_green-2.0*avg_blue)/(avg_nir+avg_green+2.0*avg_blue)

# Function to calculate ARI_2
def calculate_ari_2(avg_nir, avg_red_edge, avg_green):
    return avg_nir*((1.0/avg_green)-(1.0/avg_red_edge))

# Function to calculate CRI_2
def calculate_cri_2(avg_red_edge, avg_green):
    return (1.0/avg_green)-(1.0/avg_red_edge)

# Function to compute height statistics
def compute_statistics(image_path):
    try:
        with rasterio.open(image_path) as src:
            data = src.read(1)
            # Exclude nodata and 0 values (assuming 0 is invalid for height)
            mask = (data != src.nodata) & (data > 0)  # Mask out nodata and 0 values
            height_values = data[mask]

            if height_values.size == 0:
                raise ValueError("No valid data in the image")

            # Compute statistics on valid height values
            statistics = {
                'Hmean': np.mean(height_values),
                'Hmedian': np.median(height_values),
                'Hmin': np.min(height_values),
                'Hmax': np.max(height_values),
                'Hstd': np.std(height_values),
                'Hp90': np.percentile(height_values, 90),
                'Hp93': np.percentile(height_values, 93),
                'Hp95': np.percentile(height_values, 95),
                'Hp98': np.percentile(height_values, 98),
                'Hp99': np.percentile(height_values, 99)
            }
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        # Set default values for statistics if processing fails
        statistics = {
            'Hmean': np.nan,  # Using np.nan instead of empty string for numerical data
            'Hmedian': np.nan,
            'Hmin': np.nan,
            'Hmax': np.nan,
            'Hstd': np.nan,
            'Hp90': np.nan,
            'Hp93': np.nan,
            'Hp95': np.nan,
            'Hp98': np.nan,
            'Hp99': np.nan
        }

    return statistics

# Iterate over each chip image name in the Excel file
for chip_name in chip_filenames:
    # Get the corresponding spectral, texture, and height image paths
    spectral_image_path = os.path.join(spectral_folder, chip_name + '.tif')
    texture_image_path = os.path.join(texture_folder, chip_name + '.tif')
    height_image_path = os.path.join(height_folder, chip_name + '.tif')

    # Load spectral image
    with rasterio.open(spectral_image_path) as src:
        spectral_bands = src.read()  # Assuming spectral bands are in the first few bands
        height, width = spectral_bands.shape[1], spectral_bands.shape[2]  # Get image dimensions

    # Load texture image (assuming 40 texture bands)
    with rasterio.open(texture_image_path) as src:
        texture_bands = src.read()

    # Load height image and compute statistics using the updated compute_statistics function
    height_statistics = compute_statistics(height_image_path)

    # Extract spectral bands (assuming 5 bands: blue, green, red, red-edge, NIR)
    blue_band = spectral_bands[0]
    green_band = spectral_bands[1]
    red_band = spectral_bands[2]
    red_edge_band = spectral_bands[3]
    nir_band = spectral_bands[4]

    # Compute plot-level average pixel values for each band
    avg_blue = np.mean(blue_band) / 65536
    avg_green = np.mean(green_band) / 65536
    avg_red = np.mean(red_band) / 65536
    avg_red_edge = np.mean(red_edge_band) / 65536
    avg_nir = np.mean(nir_band) / 65536

    # Calculate spectral indices values
    avg_ndvi = calculate_ndvi(avg_nir, avg_red)
    avg_gndvi = calculate_gndvi(avg_nir, avg_green)
    avg_rvi_1 = calculate_rvi_1(avg_nir, avg_red)
    avg_gci = calculate_gci(avg_nir, avg_green)
    avg_rgvi = calculate_rgvi(avg_red, avg_green)
    avg_dvi = calculate_dvi(avg_nir, avg_red)
    avg_savi = calculate_savi(avg_nir, avg_red)
    avg_msavi = calculate_msavi(avg_nir, avg_red)
    avg_osavi = calculate_osavi(avg_nir, avg_red)
    avg_rdvi = calculate_rdvi(avg_nir, avg_red)
    avg_tvi = calculate_tvi(avg_nir, avg_red, avg_green)
    avg_tsavi = calculate_tsavi(avg_nir, avg_red)
    avg_pvi = calculate_pvi(avg_nir, avg_red)
    avg_savi_2 = calculate_savi_2(avg_nir, avg_red)
    avg_atsavi = calculate_atsavi(avg_nir, avg_red)
    avg_ndwi = calculate_ndwi(avg_nir, avg_green)
    avg_ikaw = calculate_ikaw(avg_red, avg_blue)
    avg_srpi = calculate_srpi(avg_red, avg_blue)
    rvi_2 = calculate_rvi_2(avg_nir, avg_green)
    avg_mcari = calculate_mcari(avg_red_edge, avg_red, avg_green)
    avg_mcari_1 = calculate_mcari_1(avg_nir, avg_red, avg_green)
    avg_mcari_2 = calculate_mcari_2(avg_nir, avg_red, avg_green)
    avg_mtvi_1 = calculate_mtvi_1(avg_nir, avg_red, avg_green)
    avg_mtvi_2 = calculate_mtvi_2(avg_nir, avg_green)
    avg_r_mcari_mtvi2 = calculate_r_mcari_mtvi2(avg_nir, avg_red, avg_green)
    avg_evi = calculate_evi(avg_nir, avg_red, avg_blue)
    avg_datt = calculate_datt(avg_nir, avg_red_edge, avg_red)
    avg_dnci = calculate_dnci(avg_red_edge, avg_green)
    avg_psri = calculate_psri(avg_red_edge, avg_red, avg_green)
    avg_sipi = calculate_sipi(avg_nir, avg_red, avg_blue)
    avg_spvi = calculate_spvi(avg_nir, avg_red, avg_green)
    avg_tcari = calculate_tcari(avg_red_edge, avg_red, avg_green)
    avg_r_tcari_osavi = calculate_r_tcari_osavi(avg_nir, avg_red_edge, avg_red, avg_green)
    avg_reri = calculate_reri(avg_nir, avg_red_edge, avg_red)
    avg_ndre = calculate_ndre(avg_nir, avg_red_edge)
    avg_mtci = calculate_mtci(avg_nir, avg_red_edge, avg_red)
    avg_evi_2 = calculate_evi_2(avg_nir, avg_red)
    avg_reci = calculate_reci(avg_nir, avg_red_edge)
    avg_nexg = calculate_nexg(avg_red, avg_green, avg_blue)
    avg_ngrdi = calculate_ngrdi(avg_red, avg_green)
    avg_endvi = calculate_endvi(avg_nir, avg_green, avg_blue)
    avg_ari_2 = calculate_ari_2(avg_nir, avg_red_edge, avg_green)
    avg_cri_2 = calculate_cri_2(avg_red_edge, avg_green)
    
    # Calculate average texture values across all pixels for each of the 40 texture bands
    avg_texture_features = []
    for i in range(40):
        texture_band = texture_bands[i]
        avg_texture = np.mean(texture_band)
        avg_texture_features.append(avg_texture)

    # Combine all features (average values for spectral bands, NDVI, texture features, and height-based features)
    combined_features = [chip_name, avg_blue, avg_green, avg_red, avg_red_edge, avg_nir, avg_ndvi, avg_gndvi, avg_rvi_1, avg_gci, avg_rgvi, avg_dvi,
                         avg_savi, avg_msavi, avg_osavi, avg_rdvi, avg_tvi, avg_tsavi, avg_pvi, avg_savi_2, avg_atsavi, avg_ndwi, avg_ikaw, avg_srpi,
                         rvi_2, avg_mcari, avg_mcari_1, avg_mcari_2, avg_mtvi_1, avg_mtvi_2, avg_r_mcari_mtvi2, avg_evi, avg_datt, avg_dnci, avg_psri,
                         avg_sipi, avg_spvi, avg_tcari, avg_r_tcari_osavi, avg_reri, avg_ndre, avg_mtci, avg_evi_2, avg_reci, avg_nexg, avg_ngrdi,
                         avg_endvi, avg_ari_2, avg_cri_2] + \
                        avg_texture_features + \
                        [height_statistics['Hmean'], height_statistics['Hmedian'], height_statistics['Hmin'], 
                         height_statistics['Hmax'], height_statistics['Hstd'], height_statistics['Hp90'], 
                         height_statistics['Hp93'], height_statistics['Hp95'], height_statistics['Hp98'], 
                         height_statistics['Hp99']]

    # Append the combined features to the input_features_list
    input_features_list.append(combined_features)

    # Append the combined features to the DataFrame
    features_df.loc[len(features_df)] = combined_features

    # Get the corresponding true tiller density from the DataFrame
    true_tiller_density = df_tiller_density[df_tiller_density['Chip_Name'] == chip_name]['Tiller_Density'].values[0]
    true_tiller_densities.append(true_tiller_density)

# Save the features DataFrame to an Excel file
output_features_file = r'C:\test_td\td_2023_check.xlsx'
features_df.to_excel(output_features_file, index=False)

print(f"Features for all chips saved to {output_features_file}")

# Convert lists to numpy arrays
input_features_array = np.array(input_features_list)
true_tiller_densities_array = np.array(true_tiller_densities)

# Convert lists to numpy arrays, excluding non-numeric columns such as 'Chip_Name'
# Assuming `combined_features` contains the chip name as the first value, we slice it off.
input_features_array = np.array([features[1:] for features in input_features_list], dtype=np.float32)  # Exclude Chip_Name and ensure float type
true_tiller_densities_array = np.array(true_tiller_densities, dtype=np.float32)  # Ensure tiller densities are floats

# Check if any NaN or invalid values are in the input_features_array
if np.isnan(input_features_array).any():
    print("Warning: NaN values detected in input features.")
if np.isnan(true_tiller_densities_array).any():
    print("Warning: NaN values detected in true tiller densities.")

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(input_features_array, true_tiller_densities_array, test_size=0.2, random_state=42)

# Reshape the input for 1D CNN (samples, time steps, features)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build the 1D CNN model
def build_1d_cnn(input_shape):
    model = Sequential()
    model.add(Conv1D(64, kernel_size=2, activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Conv1D(64, kernel_size=2, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))  # Output layer for regression (predicting tiller density)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# Build and compile the model
input_shape = (X_train.shape[1], 1)  # (number of features, 1)
model = build_1d_cnn(input_shape)

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=16, verbose=1)

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate R², MSE, RMSE, and MAE
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = calculate_rmse(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Print the results
print(f'R²: {r2}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')


Features for all chips saved to C:\test_td\td_2023_check.xlsx
Epoch 1/100


C:\Users\maruf\anaconda3\envs\keras_env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - loss: 27.5913 - val_loss: 11.5735
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 9.2637 - val_loss: 10.0084
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.9989 - val_loss: 7.1655
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 2.0203 - val_loss: 8.0741
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.5556 - val_loss: 8.6067
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.8941 - val_loss: 7.7250
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.7080 - val_loss: 6.3831
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.7417 - val_loss: 6.2954
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.6073 - val_loss: 7.0690
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.9678 - val_loss: 6.7177
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.5277 - val_loss: 5.3568
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.7453 - val_loss: 4.955